In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from os import listdir
from os.path import basename, splitext, exists, join
import cv2
import glob

In [3]:
### CROP FACE DETECTION

# load OpenCV face detector
face_cascade = cv2.CascadeClassifier('opencv-files/haarcascade_frontalface_alt.xml')


def face_detection(dataset_path, folder_name):
    k = 0
    images_list = []
    images_label = []
    labels_name = []
    for root in listdir(dataset_path):
        if (not root.startswith('.')):
            label_dir = join(dataset_path, root)
            for img_paths in glob.glob(os.path.join(label_dir, "*")):
                img = cv2.imread(img_paths)
                img_name = os.path.basename(img_paths) 
                img_name = os.path.basename(img_paths) 
                frame = cv2.resize(img, (640, 480))
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
                for (x, y, w, h) in faces:
                    face_crop = frame[y:y+h,x:x+w]  
                    os.makedirs(folder_name + '/' + root, exist_ok=True)
                    cv2.imwrite(folder_name + '/' + root + '/' +  img_name, face_crop)  
            k =+ 1

In [4]:
# Cropping faces from the images train folder
DATASET_TRAIN_PATH = 'data/train'
FOLDER_TRAIN_NAME = 'data/train_face'
os.makedirs(FOLDER_TRAIN_NAME, exist_ok=True)
face_detection(DATASET_TRAIN_PATH, FOLDER_TRAIN_NAME)

In [5]:
# Cropping faces from the images validation folder
DATASET_VAL_PATH = 'data/val'
FOLDER_VAL_NAME = 'data/val_face'
os.makedirs(FOLDER_VAL_NAME, exist_ok=True)
face_detection(DATASET_VAL_PATH, FOLDER_VAL_NAME)

In [6]:
#data transforms
data_transforms = {
    'train_face': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val_face': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train_face', 'val_face']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train_face', 'val_face']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train_face', 'val_face']}
class_names = image_datasets['train_face'].classes

use_gpu = torch.cuda.is_available()

In [8]:
METHOD_NAME = 'RESNET18' 
model = None
if METHOD_NAME == 'RESNET18':
    ## resnet18
    model = models.resnet18(pretrained=True)
if METHOD_NAME == 'DENSENET161':
    # densenet161
    model = models.densenet161(pretrained=True)
if METHOD_NAME == 'ALEXNET':
    ## alexnet
    model = models.alexnet(pretrained=True) 
if METHOD_NAME == 'VGG16':
    # vgg16
    model =  models.vgg16(pretrained=True)


if use_gpu:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [9]:
#******************************************************************
# TRAINING THE MODEL
#******************************************************************

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train_face', 'val_face']:
            if phase == 'train_face':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train_face':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val_face' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train_face Loss: 6.0877 Acc: 0.0000
val_face Loss: 3.2319 Acc: 0.0000

Epoch 1/24
----------
train_face Loss: 1.7661 Acc: 0.0000
val_face Loss: 2.6164 Acc: 0.0000

Epoch 2/24
----------
train_face Loss: 1.1587 Acc: 0.0000
val_face Loss: 1.6885 Acc: 0.0000

Epoch 3/24
----------
train_face Loss: 0.9761 Acc: 0.0000
val_face Loss: 1.4047 Acc: 0.0000

Epoch 4/24
----------
train_face Loss: 0.6522 Acc: 0.0000
val_face Loss: 1.2339 Acc: 0.0000

Epoch 5/24
----------
train_face Loss: 0.8286 Acc: 0.0000
val_face Loss: 0.8777 Acc: 0.0000

Epoch 6/24
----------
train_face Loss: 0.9910 Acc: 0.0000
val_face Loss: 1.4715 Acc: 0.0000

Epoch 7/24
----------
train_face Loss: 1.0899 Acc: 0.0000
val_face Loss: 1.3195 Acc: 0.0000

Epoch 8/24
----------
train_face Loss: 0.6441 Acc: 0.0000
val_face Loss: 1.1747 Acc: 0.0000

Epoch 9/24
----------
train_face Loss: 0.3488 Acc: 0.0000
val_face Loss: 0.9919 Acc: 0.0000

Epoch 10/24
----------
train_face Loss: 0.3653 Acc: 0.0000
val_face Lo

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [ ]:
os.makedirs('model', exist_ok=True)
torch.save(model, 'model/mymodel.pt')